In [6]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance
import pandas as pd

In [7]:
# Inisialisasi model embedding
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Hubungkan ke Qdrant
qdrant = QdrantClient(host="localhost", port=6333)
collection_name = "instru_collection"

print(f"Koneksi ke koleksi '{collection_name}' berhasil. Siap untuk query!")

Koneksi ke koleksi 'instru_collection' berhasil. Siap untuk query!


In [8]:
# Pertanyaan uji coba
query = "What is a transducer in instrumentation?"

# Buat embedding dari pertanyaan
query_vector = embedding_model.encode(query)

In [11]:
search_results = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3,
)

import textwrap

print("\nHasil pencarian paling relevan:\n")
for hit in search_results.points:
    content = hit.payload.get("page_content") or hit.payload.get("text", "")
    print(f"(score={hit.score:.3f})")
    print(textwrap.fill(content, width=100))
    print("\n" + "="*80 + "\n")


Hasil pencarian paling relevan:

(score=0.697)
A transducer converts one form of energy into another, often physical to electrical.


(score=0.697)
A transducer converts one form of energy into another, often physical to electrical.


(score=0.681)
Transducer A device that changes energy from one form to another. Transfer function An equation that
describes the relationship between the input and output of the function. Transmission The
transferring of information from one point to another. Transmitter A device that conditions the
signal received from a transducer so that it is suitable for sending to another location with
minimal loss of information. Turbine flow meter A flow-measuring device utilizing a turbine wheel.
Ultrasonic probe An


